In [ ]:
import sklearn
import pandas as pd
import time
import gc 
import numpy as np
from tqdm import tqdm_notebook as tqdm
from sklearn.model_selection import train_test_split, GroupKFold, KFold, TimeSeriesSplit
from sklearn.metrics import f1_score, accuracy_score, roc_auc_score, recall_score, precision_score, confusion_matrix
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
import lightgbm as lgb
import datetime 
import pickle
import seaborn as sns
from sklearn.neighbors import KDTree
from sklearn.preprocessing import StandardScaler
from matplotlib import pyplot as plt
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import warnings
warnings.filterwarnings("ignore")

%matplotlib inline
sns.set(color_codes=True)

def runQuery(query,connexion):
    cur = connexion.cursor()
    cur.execute(query)
    res = cur.fetchall()
    return pd.DataFrame(list(res),columns = [e[0] for e in cur.description])

class GoldenTimer:
    def __init__(self, show=True):
        self.start_time = time.time()
        self.show = show

    def time(self, print_str):
        duration = time.time() - self.start_time
        if self.show:
            print(print_str, duration)
        self.start_time = time.time()
        
def missing_fun(data):
    missing_df = data.dtypes.to_frame("type").reset_index()
    percent_missing = data.isnull().sum() * 100 / len(data)
    nunique = data.nunique(dropna=False).values
    missing_df["percent_missing"] = percent_missing.values
    missing_df["nunique"] = nunique
    missing_df = missing_df.loc[missing_df.percent_missing !=0]
    missing_df = missing_df.sort_values(by="percent_missing", ascending=False)
    missing_df["min"], missing_df["max"] = np.nan, np.nan
    missing_df.loc[missing_df["type"]!="object","min"] = missing_df.loc[missing_df["type"]!="object","index"].apply(lambda x:data[x].min())
    missing_df.loc[missing_df["type"]!="object","max"] = missing_df.loc[missing_df["type"]!="object","index"].apply(lambda x:data[x].max())
    missing_df["sample"] = missing_df["index"].apply(lambda x:data[x].value_counts(dropna=False).index.tolist())
    return missing_df

def plot_cm(y_true, y_pred, title):
    figsize=(5,5)
    y_pred = y_pred.astype(int)
    cm = confusion_matrix(y_true, y_pred, labels=np.unique(y_true))
    cm_sum = np.sum(cm, axis=1, keepdims=True)
    cm_perc = cm / cm_sum.astype(float) * 100
    annot = np.empty_like(cm).astype(str)
    nrows, ncols = cm.shape
    for i in range(nrows):
        for j in range(ncols):
            c = cm[i, j]
            p = cm_perc[i, j]
            if i == j:
                s = cm_sum[i]
                annot[i, j] = '%.1f%%\n%d/%d' % (p, c, s)
            elif c == 0:
                annot[i, j] = ''
            else:
                annot[i, j] = '%.1f%%\n%d' % (p, c)
    cm = pd.DataFrame(cm, index=np.unique(y_true), columns=np.unique(y_true))
    cm.index.name = 'Actual'
    cm.columns.name = 'Predicted'
    fig, ax = plt.subplots(figsize=figsize)
    plt.title(title)
    sns.heatmap(cm, cmap= "YlGnBu", annot=annot, fmt='', ax=ax)
    
def add_noise(series, noise_level):
    return series * (1 + noise_level * np.random.randn(len(series)))

def target_encode(trn_series=None, 
                  #tst_series=None, 
                  target=None, 
                  min_samples_leaf=1, 
                  smoothing=1,
                  noise_level=0):
    assert len(trn_series) == len(target)
    #assert trn_series.name == tst_series.name
    temp = pd.concat([trn_series, target], axis=1)
    # Compute target mean 
    averages = temp.groupby(by=trn_series.name)[target.name].agg(["mean", "count"])
    # Compute smoothing
    smoothing = 1 / (1 + np.exp(-(averages["count"] - min_samples_leaf) / smoothing))
    # Apply average function to all target data
    prior = target.mean()
    # The bigger the count the less full_avg is taken into account
    averages[target.name] = prior * (1 - smoothing) + averages["mean"] * smoothing
    averages.drop(["mean", "count"], axis=1, inplace=True)
    # Apply averages to trn and tst series
    ft_trn_series = pd.merge(
        trn_series.to_frame(trn_series.name),
        averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
        on=trn_series.name,
        how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    # pd.merge does not keep the index so restore it
    ft_trn_series.index = trn_series.index 
    """
    ft_tst_series = pd.merge(
        tst_series.to_frame(tst_series.name),
        averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
        on=tst_series.name,
        how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    ft_tst_series.index = tst_series.index
    """
    return add_noise(ft_trn_series, noise_level) #, add_noise(ft_tst_series, noise_level)

def stat_fun(data_out,outlier_cols):
    # Removing skewnewss and kurtosis using log transformation if it is above a threshold value (2)
    data_stat = pd.DataFrame()
    data_stat['numeric_column'] = outlier_cols
    skew_before = []
    skew_after = []

    kurt_before = []
    kurt_after = []

    standard_deviation_before = []
    standard_deviation_after = []

    log_transform_needed = []

    log_type = []

    for i in outlier_cols:
        skewval = data_out[i].skew()
        skew_before.append(skewval)

        kurtval = data_out[i].kurtosis()
        kurt_before.append(kurtval)

        sdval = data_out[i].std()
        standard_deviation_before.append(sdval)

        if (abs(skewval) > 2) & (abs(kurtval) > 2):
            log_transform_needed.append('Yes')

            if len(data_out[data_out[i] == 0])/len(data_out) <=0:
                log_type.append('log')
                skewvalnew = np.log(pd.DataFrame(data_out[data_out[i] > 0])[i]).skew()
                skew_after.append(skewvalnew)

                kurtvalnew = np.log(pd.DataFrame(data_out[data_out[i] > 0])[i]).kurtosis()
                kurt_after.append(kurtvalnew)

                sdvalnew = np.log(pd.DataFrame(data_out[data_out[i] > 0])[i]).std()
                standard_deviation_after.append(sdvalnew)

            else:
                log_type.append('log1p')
                skewvalnew = np.log1p(pd.DataFrame(data_out[data_out[i] >= 0])[i]).skew()
                skew_after.append(skewvalnew)

                kurtvalnew = np.log1p(pd.DataFrame(data_out[data_out[i] >= 0])[i]).kurtosis()
                kurt_after.append(kurtvalnew)

                sdvalnew = np.log1p(pd.DataFrame(data_out[data_out[i] >= 0])[i]).std()
                standard_deviation_after.append(sdvalnew)

        else:
            log_type.append('NA')
            log_transform_needed.append('No')

            skew_after.append(skewval)
            kurt_after.append(kurtval)
            standard_deviation_after.append(sdval)

    data_stat['skew_before'] = skew_before
    data_stat['kurtosis_before'] = kurt_before
    data_stat['standard_deviation_before'] = standard_deviation_before
    data_stat['log_transform_needed'] = log_transform_needed
    data_stat['log_type'] = log_type
    data_stat['skew_after'] = skew_after
    data_stat['kurtosis_after'] = kurt_after
    data_stat['standard_deviation_after'] = standard_deviation_after
    
    return data_stat

def outlier_fun(data_out,data_stat,outlier_cols):
    outlier_df = pd.DataFrame(columns=["feature", "log", "count", "inc_count", "lower","upper"])
    outlier_df["feature"] = outlier_cols
    card_list = []
    cols_obs = []
    out_count = 0
    for colname in outlier_cols:
        if data_stat.loc[data_stat["numeric_column"] == colname, "log_transform_needed"].values=="Yes":
            data_temp = data_out[["cc_nbr",colname]]
            if data_stat.loc[data_stat["numeric_column"] == colname, "log_type"].values=="log":
                data_temp[colname + "_log"] = np.log(data_temp[colname]) 
                temp_card = data_temp.loc[(np.abs(sp.stats.zscore(data_temp[colname+"_log"])) > 3),"cc_nbr"].values.tolist()
                lower = data_temp.loc[(sp.stats.zscore(data_temp[colname+"_log"]) < -3),colname].max()
                upper = data_temp.loc[(sp.stats.zscore(data_temp[colname+"_log"]) > 3),colname].min()
                log = "log"
                card_list.extend(temp_card)
                #print(f"Log transformation made to {colname} column ; There are {len(temp_card)} outliers")
            if data_stat.loc[data_stat["numeric_column"] == colname,"log_type"].values=="log1p":
                data_temp[colname + "_log1p"] = np.log1p(data_temp[colname]) 
                temp_card = data_temp.loc[(np.abs(sp.stats.zscore(data_temp[colname+"_log1p"])) > 3),"cc_nbr"].values.tolist()
                lower = data_temp.loc[(sp.stats.zscore(data_temp[colname+"_log1p"]) < -3),colname].max()
                upper = data_temp.loc[(sp.stats.zscore(data_temp[colname+"_log1p"]) > 3),colname].min()
                log = "log1p"
                card_list.extend(temp_card)
                #print(f"Log1p transformation made to {colname} column ; There are {len(temp_card)} outliers")
        else:
            data_temp = data_out[["cc_nbr",colname]]
            temp_card = data_temp.loc[(np.abs(sp.stats.zscore(data_temp[colname])) > 3),"cc_nbr"].values.tolist()
            lower = data_temp.loc[(sp.stats.zscore(data_temp[colname]) < -3),colname].max()
            upper = data_temp.loc[(sp.stats.zscore(data_temp[colname]) > 3),colname].min()
            log = "no"
            card_list.extend(temp_card)
            #print(f"No Log transformation made to {colname} column ; There are {len(temp_card)} outliers")
        if len(temp_card)>5000:
            cols_obs.append(colname)
        inc_count = len(set(card_list)) - out_count
        outlier_df.loc[outlier_df["feature"]==colname,["log","count", "inc_count","lower","upper"]] = [log,len(temp_card), inc_count, lower, upper]
        out_count = len(set(card_list))
    print(f"There are {len(set(card_list))} outlier Credit Cards")
    return outlier_df, card_list

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score
import plotly
import plotly.graph_objs as go
import warnings
import os
import plotly.io as pio
from sklearn.cluster import KMeans
import numpy as np

warnings.filterwarnings('ignore')

In [2]:
def clear_file(filename='single_factor_analysis.html'):
    with open(filename, mode = "w", encoding="utf-8") as file:
        file.write("")

In [5]:
def c_value(df, target='M6_F', org_feature='ResearchVar', rank_feature='RankVar'):
    try:
        nnull_data = df[~df[org_feature].isnull()].copy()
        c_org = roc_auc_score(nnull_data[target],nnull_data[org_feature])
        c_ranked = roc_auc_score(nnull_data[target],nnull_data[rank_feature])
        if c_org <= 0.5:
            c_org = 1-c_org
        if c_ranked <= 0.5:
            c_ranked = 1-c_ranked
        return c_org, c_ranked
    except:
        return 0, 0

In [6]:
def information_value(df, target='M6_F', org_feature='ResearchVar', rank_feature ='RankVar', rank_null_tr=-1, id_column="TALEP_NO"):
    try:
        tr_data = df.copy()
        tr_data[rank_feature] = tr_data[rank_feature].fillna(rank_null_tr)
    
        pv = tr_data.pivot_table(values=id_column, index=rank_feature, columns=target,aggfunc='count')
        pv.columns=['good','bad']
        num_good = pv.sum(axis=0)['good']
        num_bad = pv.sum(axis=0)['bad']
        pv['pc_good'] = pv['good']/num_good
        pv['pc_bad'] = pv['bad']/num_bad
        pv['woe']=np.log(pv['pc_good']/pv['pc_bad'])
        pv['iv'] = (pv['pc_good'] - pv['pc_bad']) * pv['woe']
        iv = pv['iv'].sum()
        return iv
    except:
        return 0

In [22]:
exec(open('./single_factor_analysis/sfa_code.py').read())

### Read Data & Feature Dictionary

In [ ]:
def fill_nas(main, dtypes_df):
    for col in dtypes_df.feature:
        if col in cat_features:
            main[col].fillna("Missing",inplace=True)
        else:
            main[col] = main[col].apply(lambda x: x if (x!=None) and (x!=np.nan) and (np.isnan(x)!=True) else main[col].mean())
    return main

In [ ]:
catalog = pd.read_csv("datathon_yedek_parca_catalog.csv",sep="|")
orders = pd.read_csv("datathon_yedek_parca_orders.csv",sep="|")
submission = pd.read_csv("sample_submission.csv",sep="|")
df = orders.merge(catalog,"left","part_id")

In [ ]:
# preprocessing
df["date"] = pd.to_datetime(df["date"])
df["stock_card_create_date"] = pd.to_datetime(df["stock_card_create_date"])
df["period"] = pd.to_datetime(df["date"]).dt.to_period('m')

df["part_season"] = df["part_season"].map({'Yaz Mevsimsel':1,'Kış Mevsimsel':0}).fillna(-1)
df["min_date"] = df.groupby("part_id")["date"].transform("min")
df["stock_card_create_date"] = df[["min_date","stock_card_create_date"]].min(axis = 1)
del df["min_date"]

# period = df[["period"]].drop_duplicates().sort_values("period").reset_index(drop=True)
period = pd.concat([df[["period"]].drop_duplicates().sort_values("period"),submission[["period"]].drop_duplicates()]).reset_index(drop=True)
period["period"] = period["period"].astype("period[M]")

ids = df[['part_id', 'firm_id', 'part_definition_id','part_product_class_id', 
            'common_part_catalog_id','preferred_supplier_id', 'part_family_id',
            'stock_card_create_date','part_season']].drop_duplicates()

period['key'] = 0
ids['key'] = 0
join = period.merge(ids, how='outer')

del join["key"], period, ids
gc.collect()

In [ ]:
data = df.groupby(['part_id', 'firm_id', 'part_definition_id','part_product_class_id', 
                   'common_part_catalog_id','preferred_supplier_id', 'part_family_id',
                   'stock_card_create_date','part_season', 'period'])["order_quantity"].agg(["sum"])\
                    .reset_index().rename({"sum":'order_quantity'},axis=1)

data["min_period"] = data.groupby(["part_id","firm_id"])["period"].transform("min")
print(data.shape)

data = join.merge(data,"left",['part_id', 'firm_id', 'part_definition_id','part_product_class_id', 
                               'common_part_catalog_id','preferred_supplier_id', 'part_family_id',
                               'stock_card_create_date','part_season', 'period'])

data["min_period"] = data.groupby(["part_id","firm_id"])["min_period"].apply(lambda x: x.fillna(method="bfill").fillna(method="ffill"))
data["order_quantity"] = data["order_quantity"].fillna(0)
print(data.shape)

data = data[data["period"]>=data["min_period"]]
del data["min_period"]

In [ ]:
data["time"] = data["period"].rank(method = "dense")

df = df.sort_values(["part_id","date"]).reset_index(drop=True)
df["time"] = df["period"].rank(method = "dense")
df["time_diff"] = df.groupby(["part_id"])["time"].diff()
time_diff_mean = df.groupby(["part_id"])["time_diff"].mean().to_dict()
time_diff_std = df.groupby(["part_id"])["time_diff"].std().to_dict()
time_diff_min = df.groupby(["part_id"])["time_diff"].min().to_dict()
time_diff_max = df.groupby(["part_id"])["time_diff"].max().to_dict()

df = df.sort_values(["part_id","firm_id","date"]).reset_index(drop=True)
df["time_diff_firm"] = df.groupby(["part_id","firm_id"])["time"].diff()

time_diff_firm_mean = df.groupby(["part_id"])["time_diff_firm"].mean().to_dict()
time_diff_firm_std = df.groupby(["part_id"])["time_diff_firm"].std().to_dict()
time_diff_firm_min = df.groupby(["part_id"])["time_diff_firm"].min().to_dict()
time_diff_firm_max = df.groupby(["part_id"])["time_diff_firm"].max().to_dict()

df["order_quantity_first"] = df.groupby(["part_id","firm_id"])["order_quantity"].transform("first")
order_quantity_first_mean = df.groupby(["part_id"])["order_quantity_first"].mean().to_dict()
order_quantity_first_std = df.groupby(["part_id"])["order_quantity_first"].std().to_dict()

df["order_quantity_last"] = df.groupby(["part_id","firm_id"])["order_quantity"].transform("last")
order_quantity_last_mean = df.groupby(["part_id"])["order_quantity_last"].mean().to_dict()
order_quantity_last_std = df.groupby(["part_id"])["order_quantity_last"].std().to_dict()

df["order_quantity_min"] = df.groupby(["part_id","firm_id"])["order_quantity"].transform("min")
order_quantity_min_mean = df.groupby(["part_id"])["order_quantity_min"].mean().to_dict()
order_quantity_min_std = df.groupby(["part_id"])["order_quantity_min"].std().to_dict()

df["order_quantity_max"] = df.groupby(["part_id","firm_id"])["order_quantity"].transform("max")
order_quantity_max_mean = df.groupby(["part_id"])["order_quantity_max"].mean().to_dict()
order_quantity_max_std = df.groupby(["part_id"])["order_quantity_max"].std().to_dict()

data = data.sort_values(["part_id","firm_id","period"]).reset_index(drop=True)
data.head()

In [ ]:
data["last_time"] = data.loc[data["order_quantity"]>0,"time"]
data["last_time"] = data.groupby(["part_id","firm_id"])["last_time"].apply(lambda x: x.fillna(method="ffill"))
data["last_time"] = data["time"] - data["last_time"]

last_time_mean = data.groupby(["part_id","period"])["last_time"].mean().reset_index().rename({"last_time":"last_time_mean"},axis=1)
last_time_std = data.groupby(["part_id","period"])["last_time"].std().reset_index().rename({"last_time":"last_time_std"},axis=1)

train = data.groupby(['part_id', 'part_definition_id','part_product_class_id', 'common_part_catalog_id',
                      'preferred_supplier_id', 'part_family_id','stock_card_create_date','part_season', 'period'])["order_quantity"].agg(["sum"])\
                     .reset_index().rename({"sum":'order_quantity'},axis=1).sort_values(["part_id","period"])
train["time"] = train["period"].rank(method = "dense")

In [ ]:
for i in tqdm([1,2,3,6]):
    train[f"order_quantity_lag{i}"] = train.groupby("part_id")["order_quantity"].shift(i)
    
    for j in [6,12]:
        train[f'order_quantity_lag{i}_w{j}_mean'] = train.groupby(['part_id'])["order_quantity"].transform(lambda x: x.shift(i).rolling(j).mean())
        train[f'order_quantity_lag{i}_w{j}_std'] = train.groupby(['part_id'])["order_quantity"].transform(lambda x: x.shift(i).rolling(j).std())
        
train["order_quantity_lag1_w12_w6_ratio"] = train["order_quantity_lag1_w12_mean"]/train["order_quantity_lag1_w6_mean"]
train["order_quantity_lag2_w12_w6_ratio"] = train["order_quantity_lag2_w12_mean"]/train["order_quantity_lag2_w6_mean"]
train["order_quantity_lag3_w12_w6_ratio"] = train["order_quantity_lag3_w12_mean"]/train["order_quantity_lag3_w6_mean"]
train["order_quantity_lag6_w12_w6_ratio"] = train["order_quantity_lag6_w12_mean"]/train["order_quantity_lag6_w6_mean"]

In [ ]:
for col in tqdm(['part_product_class_id', 'part_family_id']):
    train[f"order_{col}"] = train.groupby([col,"period"])["order_quantity"].transform("sum")
    for i in [1,2,3]:
        train[f"order_{col}_lag{i}"] = train.groupby(col)[f"order_{col}"].shift(i)
        
    del train[f"order_{col}"]

train = train.sort_values(["part_id","period"])
train["order_normalized"] = train.groupby(["part_id"])["order_quantity"].transform("cumsum")/(train["time"] - train.groupby(["part_id"])["time"].transform("min") + 1)
for i in tqdm([1,2,3]):
    train[f"order_normalized_lag{i}"] = train.groupby("part_id")["order_normalized"].shift(i)
    
del train["order_normalized"]

In [ ]:
for col in ["preferred_supplier_id","common_part_catalog_id"]:
    train[f"{col}_freq"] = train.groupby([col])["time"].transform("count")/len(train)

parts = train.groupby(["part_id","time"])["order_quantity"].agg(["sum"]).reset_index().pivot_table("sum",["part_id"],"time").fillna(0).reset_index()
time_cols = parts.columns[1:].tolist()
parts["min"] = parts[time_cols].min(axis=1)
parts["max"] = parts[time_cols].max(axis=1)
parts["median"] = parts[time_cols].median(axis=1)

X = parts.iloc[:,1:].values
scaler = StandardScaler()
X = scaler.fit_transform(X)

tree = KDTree(X, leaf_size=2)              # doctest: +SKIP
dist, ind = tree.query(X, k=4)                # doctest: +SKIP
print(ind)  # indices of 3 closest neighbors

In [ ]:
parts["neighbor1"] = parts["neighbor2"] = parts["neighbor3"] = 0
for i in range(5000):
    parts["neighbor1"].iloc[i] = parts["part_id"].iloc[ind[i][1]]
    parts["neighbor2"].iloc[i] = parts["part_id"].iloc[ind[i][2]]
    parts["neighbor3"].iloc[i] = parts["part_id"].iloc[ind[i][3]]   
    
for col in ["neighbor1","neighbor2","neighbor3"]:
    if col in train.columns: del train[col]
train = train.merge(parts[["part_id","neighbor1","neighbor2","neighbor3"]],"left","part_id")

In [ ]:
train["time_diff_mean"] = train["part_id"].map(time_diff_mean)
train["time_diff_std"] = train["part_id"].map(time_diff_std)
train["time_diff_firm_mean"] = train["part_id"].map(time_diff_firm_mean)
train["time_diff_firm_std"] = train["part_id"].map(time_diff_firm_std)

firm_count_mean = df.groupby(["part_id","firm_id"])["time"].count().reset_index().groupby("part_id")["time"].mean().to_dict()
train["firm_count_mean"] = train["part_id"].map(firm_count_mean)

In [ ]:
firm_nunique = df.groupby(["part_id","period"])["firm_id"].nunique().reset_index().rename({"firm_id":"firm_nunique"},axis = 1)
train = train.merge(firm_nunique,"left",["part_id","period"])
for col in ["firm_nunique"]:
    for i in tqdm([1,2,3]):
        train[f"{col}_lag{i}"] = train.groupby("part_id")[col].shift(i)
        
    del train[col]

In [ ]:
train["order_quantity_first_mean"] = train["part_id"].map(order_quantity_first_mean)
train["order_quantity_first_std"] = train["part_id"].map(order_quantity_first_std)

train["order_quantity_last_mean"] = train["part_id"].map(order_quantity_last_mean)
train["order_quantity_last_std"] = train["part_id"].map(order_quantity_last_std)

train["order_quantity_first_last"] = train["order_quantity_first_mean"] - train["order_quantity_last_mean"]

train["order_quantity_min_mean"] = train["part_id"].map(order_quantity_min_mean)
train["order_quantity_min_std"] = train["part_id"].map(order_quantity_min_std)

train["order_quantity_max_mean"] = train["part_id"].map(order_quantity_max_mean)
train["order_quantity_max_std"] = train["part_id"].map(order_quantity_max_std)

train["order_quantity_min_max"] = train["order_quantity_min_mean"] / train["order_quantity_max_mean"]
del train["order_quantity_min_mean"], train["order_quantity_min_std"]

train = train.merge(last_time_mean,"left",["part_id","period"]).merge(last_time_std,"left",["part_id","period"])
for col in ["last_time_mean","last_time_std"]:
    for i in tqdm([1,2,3]):
        train[f"{col}_lag{i}"] = train.groupby("part_id")[col].shift(i)
        
    del train[col]

In [ ]:
# train.to_csv('train.csv')

### SFA - Continuous Variables

In [ ]:
# main = pd.read_csv('./data/model_data_jan.csv')
# main.reset_index(inplace=True)
# main.rename(columns={"index":"id","Churn.30":"TARGET"},inplace=True)
# main = fill_nas(main,sfa_owner_df)

In [8]:
clear_file("single_factor_analysis_jan.html")

In [9]:
train = pd.read_csv('./train.csv')

In [17]:
cont_features = ['time_diff_std','time_diff_mean','order_normalized_lag3']

In [11]:
train = train[train.time.isin([39,38,37])==False]
train.reset_index(inplace=True)
train.rename({'index':'id'},axis=1,inplace=True)
train.rename({'order_quantity':'TARGET'},axis=1,inplace=True)

In [13]:
main = train

In [20]:
main['order_normalized_lag3'].fillna(0, inplace=True)

In [23]:
c_values = []
graph_stats_dfs_cont = []
# with open('single_factor_analysis_jan.html', mode = "a", encoding="utf-8") as file:
#     file.write("<b>Continuous Variables Analysis")
    
for feat in cont_features:
    print("="*3+" "+feat+" "+"="*3)
    graph_stats, stats_df, minimum, maximum = sfa_cont(main, feat, 'TARGET', 6,id_column="id")

    exec(open('./single_factor_analysis/draw_table.py').read())
    pio.write_image(fig0, './single_factor_analysis/images/table/'+feat+'_table.png')

    exec(open('./single_factor_analysis/draw_graph.py').read())
    pio.write_image(fig, './single_factor_analysis/images/graph/'+feat+'_graph.png')

    c_values.append([feat,stats_df.iloc[0]['org c-value']])
    graph_stats_dfs_cont.append([feat,graph_stats])

    # file.write(pio.to_html(fig, default_width = '50%', default_height = '50%'))
    # file.write(pio.to_html(fig0))
    print ('OK for '+feat)

c_values_df = pd.DataFrame(c_values, columns=['feature','c_value'])
# c_values_df.sort_values(by='c_value',ascending=False).to_excel('./cont_feature_ranking.xlsx')
c_values_df.sort_values(by='c_value',ascending=False,inplace=True)
# file.write("<b>C-Values Table</b>")
# file.write(c_values_df.to_html())
print('Finished Continuous SFA')
graph_stats_dfs_cont = pd.DataFrame(graph_stats_dfs_cont,columns=['feature','stats_df'])

=== time_diff_std ===
Number of not-null observations: 167059
OK for time_diff_std
=== time_diff_mean ===
Number of not-null observations: 167059
OK for time_diff_mean
=== order_normalized_lag3 ===
Number of not-null observations: 167059
OK for order_normalized_lag3
Finished Continuous SFA


### SFA - Categoric Variables

In [ ]:
information_values = []
graph_stats_dfs_cat=[]
# with open('zipcode_sfa.html', mode = "a", encoding="utf-8") as file:
#     file.write("<b>Continuous Variables Analysis")

for feat in cat_features:
#     if feat.find("zip_code")!=-1 or feat.find("locality")!=-1:
        # print("="*3+" "+feat+" "+"="*3)
    graph_stats, stats_df = sfa_cat(main, feat, 'TARGET',id_column="id")

    exec(open('./single_factor_analysis/draw_table_cat.py').read())
    pio.write_image(fig0, './single_factor_analysis/images/table/'+feat+'_table.png')

    exec(open('./single_factor_analysis/draw_graph_cat.py').read())
    pio.write_image(fig, './single_factor_analysis/images/graph/'+feat+'_graph.png')
    information_values.append([feat,stats_df.iloc[0]['iv']])
    graph_stats_dfs_cat.append([feat,graph_stats])

#             file.write(pio.to_html(fig, default_width = '50%', default_height = '50%'))
#             file.write(pio.to_html(fig_cat))
    print ('OK for '+feat)
        
information_values_df = pd.DataFrame(information_values, columns=['feature','iv'])
information_values_df.sort_values(by='iv',ascending=False,inplace=True)
    # information_values_df.sort_values(by='iv',ascending=False).to_excel('./cont_feature_ranking.xlsx')
#     file.write("<b>IV Table</b>")
#     file.write(information_values_df.to_html())
print('Finished Categorical SFA')
graph_stats_dfs_cat = pd.DataFrame(graph_stats_dfs_cat,columns=['feature','stats_df'])

In [ ]:
def find_familia(x):
    iy = x.find("transformed")
    ix = x.find("clustered")
    if x.find("transformed")!=-1 or x.find("clustered")!=-1:
        return x[:(max(ix,iy)-1)]
    else:
        return x

In [ ]:
information_values_df["familia"] = information_values_df.feature.apply(find_familia)

In [ ]:
information_values_df.to_excel("ivs_jan_2.xlsx")

### Apply Clustering (Categorical Features)

In [ ]:
def get_clustering_scheme(graph_stats_dfs_cat, steps=2, max_clusters= 10):
    clustering_scheme = {}
    for i in graph_stats_dfs_cat.index:
        tmp = graph_stats_dfs_cat.loc[i].stats_df
        n=tmp.shape[0]
        if n>=max_clusters:
            n=max_clusters
        if n>2:
            clustering_scheme[tmp.index.name] = list(range(2,n,steps))
    return clustering_scheme

In [ ]:
def apply_kmeans(graph_stats_dfs_cat, feature, n_clusters=2):
    try:
        tmp = graph_stats_dfs_cat[graph_stats_dfs_cat.feature==feature].iloc[0].stats_df
        X = np.array([[e,f] for e,f in zip([1]*tmp.shape[0],tmp.target_ratio.values)])
        kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(X)
        tmp["category"]=kmeans.labels_
        cluster_map = {}
        for cat in tmp.category.unique():
            cluster_map[cat]=tmp[tmp['category']==cat].index.tolist()
        print(feature+" splitted to "+str(n_clusters)+" categories.")
        return cluster_map
    except:
        print(feature+" can't be splitted to "+str(n_clusters)+" categories.")

In [ ]:
def find_cluster(x,cluster_map):
    for key,value in cluster_map.items():
        if(x in value):
            return key
    print("cant find "+str(x)+" in cluster_map")
    return -1

In [ ]:
def apply_clusters_to_data(main,graph_stats_dfs_cat,feature,n_clusters=5,suffix="_clustered"):
    cluster_map = apply_kmeans(graph_stats_dfs_cat,feature, n_clusters)
    main[feature+suffix+"_"+str(len(cluster_map.keys()))] = main[feature].apply(find_cluster,args=(cluster_map,))
    return main

In [ ]:
clustering_scheme = get_clustering_scheme(graph_stats_dfs_cat,steps=3,max_clusters=12)

In [ ]:
clustering_scheme

In [ ]:
for feature in clustering_scheme.keys():
    for i in clustering_scheme[feature]:
        print(feature + " - "+ str(i))
        main = apply_clusters_to_data(main, graph_stats_dfs_cat, feature, n_clusters=i)

### Apply Threshold (Continuous Features)

In [ ]:
main["total_segundos_espera_transformed"] = (main["total_segundos_espera"]>25)*1
main["total_segundos_conversacion_transformed"] = (main["total_segundos_conversacion"]>75)*1
main["ranking_transformed"] = (main["ranking"]==1)*1
main["tariff_ds_fibra_quota_transformed"] = (main["tariff_ds_fibra_quota"]==300)*1

### Export Final Data

In [ ]:
main.to_csv('./data/model_data_jan.csv',index=False)